In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard, ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.
/Users/a1900001.1/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def network(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(
        32, kernel_size=3, padding="same",
        input_shape=input_shape, activation="relu"
        ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # classify the class by fully-connected layers
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    return model

In [3]:
class CIFAR10Dataset():

    def __init__(self):
        self.image_shape = (32, 32, 3)
        self.num_classes = 10

    def get_batch(self):
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()

        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in
                           [y_train, y_test]]

        return x_train, y_train, x_test, y_test

    def preprocess(self, data, label_data=False):
        if label_data:
            data = keras.utils.to_categorical(data, self.num_classes)
        else:
            data = data.astype("float32")
            data /= 255  
            shape = (data.shape[0],) + self.image_shape 
            data = data.reshape(shape)

        return data

In [9]:
class Trainer():

    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(
            loss=loss, optimizer=optimizer, metrics=["accuracy"]
            )
        self.verbose = 1
        logdir = "logdir_cifar10_net"
        self.log_dir = os.path.join(os.path.dirname('__file__'), 'logdir')
        self.model_file_name = "model_file.hdf5"

    def train(self, x_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)  
        os.mkdir(self.log_dir)

        model_path = os.path.join(self.log_dir, self.model_file_name)
        self._target.fit(
            x_train, y_train,
            batch_size=batch_size, epochs=epochs,
            validation_split=validation_split,
            callbacks=[
                TensorBoard(log_dir=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True)
            ],
            verbose=self.verbose
        )

In [11]:
dataset = CIFAR10Dataset()

model = network(dataset.image_shape, dataset.num_classes)

x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0] )
print("Test accuacy:", score[1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/12
40000/40000 [==============================] - 73s 2ms/step - loss: 1.7858 - acc: 0.3623 - val_loss: 1.4229 - val_acc: 0.5100
Epoch 2/12
40000/40000 [==============================] - 91s 2ms/step - loss: 1.3428 - acc: 0.5230 - val_loss: 1.1877 - val_acc: 0.5927
Epoch 3/12
40000/40000 [==============================] - 107s 3ms/step - loss: 1.1688 - acc: 0.5883 - val_loss: 1.0530 - val_acc: 0.6393
Epoch 4/12
40000/40000 [==============================] - 112s 3ms/step - loss: 1.0547 - acc: 0.6307 - val_loss: 1.0143 - val_acc: 0.6459
Epoch 5/12
40000/40000 [==============================] - 115s 3ms/step - loss: 0.9736 - acc: 0.6594 - val_loss: 0.9526 - val_acc: 0.6702
Epoch 6/12
40000/40000 [==============================] - 117s 3ms/step - loss: 0.9114 - acc: 0.6819 - val_loss: 0.9753 - val_acc: 0.6618
Epoch 7/12
40000/40000 [==============================] - 120s 3ms/step - loss: 0.8488 - acc: 0.7052 - val_loss: 1.0468 - va